In [2]:
!pip install langdetect
!pip install contractions

     ---------------------------------------- 0.0/981.5 kB ? eta -:--:--
     ------------- ------------------------ 348.2/981.5 kB 7.2 MB/s eta 0:00:01
     ----------------------------------- -- 911.4/981.5 kB 9.6 MB/s eta 0:00:01
     -------------------------------------- 981.5/981.5 kB 8.8 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993253 sha256=3edd36ce5a9549c33da8e0095baf536340570431c05ccddc28f736ec2a573d4a
  Stored in directory: c:\users\ray\appdata\local\pip\cache\wheels\13\c7\b0\79f66658626032e78fc1a83103690ef6797d551cb22e56e734
Successfully built langdetect
     ---------------------------------------- 0.0/289.9 kB ? eta -:--:--
     -------------------------------------- 289.9/289.9 kB 9.0 MB/s eta 0:00:00


In [3]:
# split sentences
import csv
import re
import pandas as pd
import numpy as np
from langdetect import detect
import string
import matplotlib.pyplot as plt
from glob import glob
#text preprocess file
import unicodedata
import contractions         
import re
import nltk
from nltk import sent_tokenize
from nltk.stem import LancasterStemmer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm
from sklearn.model_selection import train_test_split

nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Ray\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Ray\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Ray\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Ray\AppData\Roaming\nltk_data...


True

# Load review data

In [8]:
# all review data
import os
folder_path = './SentenceMultiple'
file_names = os.listdir(folder_path)
file_names

['new_sentence-multiple-aspect-Arab Street_updated.csv',
 'new_sentence-multiple-aspect-ArtScience Museum at Marina Bay Sands_updated.csv',
 'new_sentence-multiple-aspect-Buddha Tooth Relic Temple and Museum_updated.csv',
 'new_sentence-multiple-aspect-Clarke Quay_updated.csv',
 'new_sentence-multiple-aspect-Gardens by the Bay_updated.csv',
 'new_sentence-multiple-aspect-Jurong Bird Park_updated.csv',
 'new_sentence-multiple-aspect-Maxwell Food Centre_updated.csv',
 'new_sentence-multiple-aspect-Merlion Park_updated.csv',
 'new_sentence-multiple-aspect-Mustafa Centre_updated.csv',
 'new_sentence-multiple-aspect-National Museum of Singapore_updated.csv',
 'new_sentence-multiple-aspect-National Orchid Garden_updated.csv',
 'new_sentence-multiple-aspect-Night Safari_updated.csv',
 'new_sentence-multiple-aspect-Orchard Road_updated.csv',
 'new_sentence-multiple-aspect-River Wonders_updated.csv',
 'new_sentence-multiple-aspect-Sands Skypark Observation Deck_updated.csv',
 'new_sentence-mult

In [6]:
# load all places
amusements = pd.read_excel('./20_places.xlsx')['Place']

print(f"Number of amusements: {len(amusements)}")
amusements

Number of amusements: 20


0                               Arab Street
1      Buddha Tooth Relic Temple and Museum
2                               Clarke Quay
3                        Gardens by the Bay
4                          Jurong Bird Park
5     ArtScience Museum at Marina Bay Sands
6                       Maxwell Food Centre
7                              Merlion Park
8                            Mustafa Centre
9              National Museum of Singapore
10                   National Orchid Garden
11                             Night Safari
12                             Orchard Road
13                            River Wonders
14           Sands Skypark Observation Deck
15                          Singapore Flyer
16                          Singapore River
17                            Singapore Zoo
18                Singapore Botanic Gardens
19        Singapore Mass Rapid Transit SMRT
Name: Place, dtype: object

# Process Review Data

In [9]:
def split_sentences(text):
    return re.split(r'[.!?;]', text)

def replace_contractions(text):
    """Replace contractions in string of text"""
    return contractions.fix(text)


def remove_non_ascii(text):
    # Define a list of ASCII characters
    ascii_letters = string.ascii_letters + string.digits + string.punctuation + ' '
    
    # Filter out non-ASCII characters
    ascii_text = ''.join(filter(lambda x: x in ascii_letters, text))
    
    return ascii_text

def remove_special_characters(text):
    # Define the pattern to match special characters
    pattern = r'[^a-zA-Z\s]'
    # Remove the special characters
    filtered_text = re.sub(pattern, '', text)
    return filtered_text

def replace_numbers(words):
    """Replace all interger occurrences in list of tokenized words with textual representation"""
    p = inflect.engine()
    new_words = []
    for word in words:
        if word.isdigit():
            new_word = p.number_to_words(word)
            new_words.append(new_word)
        else:
            new_words.append(word)
    return new_words

def remove_stopwords(text):
    # Define a list of stopwords for the English language
    stop_words = set(stopwords.words('english'))
    # Split the text into words
    words = text.split()
    # Remove the stopwords
    filtered_words = [word for word in words if word.lower() not in stop_words]
    # Join the filtered words back into a text string
    filtered_text = ' '.join(filtered_words)
    return filtered_text

wnl = nltk.WordNetLemmatizer()

def pos_tagger(review):
    #print(review)
    lemma_text = []
    text = nltk.word_tokenize(review)
    pos_text = nltk.pos_tag(text)
    for word, tag in pos_text:
        if tag[0] in ['N', 'V', 'J', 'R']:
            if tag in ('NN','NNPS','NNP','NNS','RB', 'RBR', 'RBS','VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ','ADV'):
                lemma_text.append(wnl.lemmatize(word,pos= tag[0].lower()))
            elif tag in ('JJR', 'JJS','JJ'):
                lemma_text.append(wnl.lemmatize(word, pos = 'a'))
            else:
                lemma_text.append(word)
    return lemma_text

def lemmatization_tagger(text):
   
    clean_text = pos_tagger(text)
    lemma_text = ' '.join(clean_text)
    return lemma_text

def detect_lang(text):
    try:
        lang = detect(x)
    except:
        lang = 'others'
    return lang

def preprocess(text):
    text = replace_contractions(text)
    text = remove_non_ascii(text)
    text = text.lower()
    text = remove_stopwords(text)
    text = remove_special_characters(text)
    text = lemmatization_tagger(text)
    return text.strip()

# Find Aspects

In [10]:
# load aspect terms

def get_aspects_df(df, place):
  aspect_df = df.loc[df['Place'] == place][['Aspect', 'Aspect Terms']]
  aspect_term_data =  [data.replace('\'', '').replace('[', '').replace(']', '').split(',') for data in aspect_df['Aspect Terms']]
  # pattern = r"['](.*?)[']"
  # aspect_term_data = [re.findall(pattern, data) for data in  aspect_df['Aspect Terms']]
  aspect_df['Aspect Terms'] = aspect_term_data
  return aspect_df

aspect_term_df = pd.read_excel('./aspects.xlsx')[['Place', 'Aspect', 'Aspect Terms']]


In [11]:
set(aspect_term_df['Place'])

{'Arab Street',
 'ArtScience Museum at Marina Bay Sands',
 'Buddha Tooth Relic Temple and Museum',
 'Clarke Quay',
 'Gardens by the Bay',
 'Jurong Bird Park',
 'Maxwell Food Centre',
 'Merlion Park',
 'Mustafa Centre',
 'National Museum of Singapore',
 'National Orchid Garden',
 'Orchard Road',
 'River Wonders',
 'Sands Skypark Observation Deck',
 'Singapore Botanic Gardens',
 'Singapore Flyer',
 'Singapore Mass Rapid Transit SMRT',
 'Singapore River',
 'Singapore Zoo'}

In [12]:
df = get_aspects_df(aspect_term_df, "Arab Street")
df

,Aspect,Aspect Terms
0,Culture & Religion,"[street, lane, muslim, soak, sultan, neig..."
1,Shopping,"[shisha, textile, ware, silk, shop, rug, ..."
2,Food,"[restuarant, pub, dining, food, cuisine, ..."


In [13]:
# load common aspects
common_aspects = {}
common_aspects["Transportation & Accessibility"] =  ['destination', 'bus', 'map', 'transit', 'metro', 'transport', 'transportation', 'network', 'credit', 'station', 'smrt','train', 'pass', 'subway', 'route', 'journey', 'passenger', 'direction']
common_aspects["Cleanliness"] = ['sanitation', 'hygiene', 'tidiness', 'orderliness', 'neatness', 'clean', 'spotless', 'dirt', 'dust', 'litter', 'rubbish', 'stain', 'mess', 'trash', 'grime', 'smudge', 'spill']
common_aspects["Value for Money"] = ['price', 'cost', 'fee', 'discount', 'rate', 'value', 'expensive', 'cheap', 'affordable', 'budget', 'deal', 'discount', 'savings', 'economy', 'bargain', 'investment', 'worth']
common_aspects.keys()

dict_keys(['Transportation & Accessibility', 'Cleanliness', 'Value for Money'])

In [40]:
def find_aspect(word, aspects, aspect_terms):
    for aspect, terms in zip(aspects, aspect_terms):
        for term in terms:
            if word == term:
                return aspect

In [60]:
amusement_n = 0 
for amusement in amusements:
    amusement_n +=1
    input_file = f'./SentenceMultiple/new_sentence-multiple-aspect-{amusement}_updated.csv'
    output_file = f'./SentenceConsolidated/consolidated-{amusement}.csv'


    print(f'{amusement_n} processing {amusement}')
    df = get_aspects_df(aspect_term_df, amusement)
    aspects = list(df['Aspect'])
    aspect_terms = list(df["Aspect Terms"])

    # add common aspects
    for common_aspect in common_aspects.keys():
        if len(aspects) < 5 and common_aspect not in aspects:

            aspects.append(common_aspect)
            aspect_terms.append(common_aspects[common_aspect])

   
    with open(input_file, 'r', encoding='utf-8', errors='replace') as infile, open(output_file, 'w', encoding='utf-8', errors='replace', newline='') as outfile:
#         fieldnames_old = reader.fieldnames
#         print(fieldnames_old)
        reader = csv.DictReader(infile)
        fieldnames = ['place', 'reviewer', 'reviewer_location', 'reviewer_contributions', 'review_rating', 'review_type', 'review_date', 'review_title', 'review_text', 'review_preprocessed']
        new_fieldnames = fieldnames + ['aspect_term', 'aspect', 'sentiment']

        writer = csv.DictWriter(outfile, fieldnames=new_fieldnames)
        writer.writeheader()

        for row in reader:
            for i in range(1, 6):
                t_header = f'target_{i}'
                if t_header in row.keys():
                    aspect_word = row[t_header]
                    if aspect_word:
                        prepprocessed_word = preprocess(aspect_word).split(" ")[0]
                        aspect_name = find_aspect(prepprocessed_word, aspects, aspect_terms)
                        if aspect_name:
                            new_row = {key: row[key] for key in fieldnames}
                            new_row['aspect_term'] = prepprocessed_word
                            new_row['aspect'] = aspect_name
                            new_row['sentiment'] = row[f'sentiment_{i}']
                            writer.writerow(new_row)
                            

1 processing Arab Street
2 processing Buddha Tooth Relic Temple and Museum
3 processing Clarke Quay
4 processing Gardens by the Bay
5 processing Jurong Bird Park
6 processing ArtScience Museum at Marina Bay Sands
7 processing Maxwell Food Centre
8 processing Merlion Park
9 processing Mustafa Centre
10 processing National Museum of Singapore
11 processing National Orchid Garden
12 processing Night Safari
13 processing Orchard Road
14 processing River Wonders
15 processing Sands Skypark Observation Deck
16 processing Singapore Flyer
17 processing Singapore River
18 processing Singapore Zoo
19 processing Singapore Botanic Gardens
20 processing Singapore Mass Rapid Transit SMRT


# Append Labeled data





In [79]:
!dir

 驱动器 C 中的卷是 Windows-SSD
 卷的序列号是 5EC1-52B9

 C:\Users\Ray\Desktop\review 的目录

2023/05/20  01:25    <DIR>          .
2023/05/19  16:50    <DIR>          ..
2023/05/19  23:36    <DIR>          .ipynb_checkpoints
2023/05/18  19:20             8,946 20_places.xlsx
2023/05/20  01:22           201,215 ABSA_fine_tune.ipynb
2023/05/19  22:27            19,749 aspects.xlsx
2023/05/20  01:25            54,238 Aspect_Consolidate_Data.ipynb
2023/05/19  01:19           158,230 aspect_label.ipynb
2023/05/20  00:15    <DIR>          Model
2023/05/19  23:02    <DIR>          SentenceConsolidated
2023/05/19  23:44    <DIR>          SentenceInfer
2023/05/20  01:22    <DIR>          SentenceInferData
2023/05/19  22:06    <DIR>          SentenceMultiple
2023/05/19  23:41    <DIR>          SentenceMultipleLabel
2023/05/18  21:32    <DIR>          SentenceTrainData
2023/05/19  21:31             5,500 Untitled.ipynb
               6 个文件        447,878 字节
              10 个目录 304,289,439,744 可用字节


In [93]:
# Load data and set labels
cols = ['place', 'reviewer', 'reviewer_location', 'reviewer_contributions',
       'review_rating', 'review_type', 'review_date', 'review_title',
       'review_text', 'review_preprocessed', 'aspect_term', 'aspect',
       ]


amusement_n = 0 
for amusement in amusements:
    amusement_n +=1
    print(f'{amusement_n} processing {amusement}')

    output_file = f'./SentenceConsolidated/consolidated-{amusement}.csv'
    
    multiple_file = f'./SentenceMultipleLabel/consolidated-{amusement}.csv'
    labeled_file = f'./SentenceTrainData/train-label-{amusement}.csv'
    inferred_file = f'./SentenceInferData/infer-label-{amusement}.csv'
    
    data_df =  pd.read_csv(multiple_file)
    df = pd.read_csv(labeled_file)
    labeled_df = df[cols].copy()
#     labeled_df['sentiment'] = df['label']
    labeled_df.loc[:,'sentiment'] = df['label']
    
    inferred_df =  pd.read_csv(inferred_file)[cols+['sentiment']]
    
    data_df = pd.concat([data_df, labeled_df, inferred_df], ignore_index=True)
    aspects =  [data.replace('\'', '').replace('[', '').replace(']', '') for data in data_df['aspect']]
    data_df['aspect'] = aspects
    print(f'total number of sentence: {data_df.shape[0]}')
    data_df.to_csv(output_file, index=False)

1 processing Arab Street
total number of sentence: 1144
2 processing Buddha Tooth Relic Temple and Museum
total number of sentence: 1693
3 processing Clarke Quay
total number of sentence: 1797
4 processing Gardens by the Bay
total number of sentence: 16278
5 processing Jurong Bird Park
total number of sentence: 1200
6 processing ArtScience Museum at Marina Bay Sands
total number of sentence: 1222
7 processing Maxwell Food Centre
total number of sentence: 3752
8 processing Merlion Park
total number of sentence: 1491
9 processing Mustafa Centre
total number of sentence: 573
10 processing National Museum of Singapore
total number of sentence: 2592
11 processing National Orchid Garden
total number of sentence: 821
12 processing Night Safari
total number of sentence: 2292
13 processing Orchard Road
total number of sentence: 3213
14 processing River Wonders
total number of sentence: 1083
15 processing Sands Skypark Observation Deck
total number of sentence: 2755
16 processing Singapore Flyer

In [94]:
total_sentences = [
    1176, 1734, 2330, 18016, 1206, 1255, 3823, 1546, 595, 2690,
    825, 2332, 3405, 1100, 2798, 5824, 510, 15422, 1010, 6641
]

sentences_with_multiple_aspects = [
    41, 60, 645, 2222, 7, 41, 691, 62, 24, 118,
    4, 74, 253, 22, 45, 61, 12, 1594, 3, 265
]
sum(total_sentences), sum(sentences_with_multiple_aspects)

(74238, 6244)